In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# 1. Compute log returns and correlation

def compute_log_returns(price_df):
    # Return log price differences.
    log_prices = np.log(price_df)
    return log_prices.diff().dropna()


def compute_correlation_matrix(returns_df):
    # Return correlation matrix of returns.
    return returns_df.corr()

# 2. MST and clustering

class UnionFind:
    def __init__(self, items):
        self.parent = {x: x for x in items}
        self.rank = {x: 0 for x in items}

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return False

        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        elif self.rank[ra] > self.rank[rb]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1

        return True


def build_mst_from_corr(corr_matrix):
    # Build an MST using distance = 1 - correlation.
    names = list(corr_matrix.columns)
    edges = []

    # collect distances
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            a, b = names[i], names[j]
            # cite the below line from AI
            c = corr_matrix.loc[a, b]
            edges.append((1 - c, a, b, c))

    edges.sort(key=lambda x: x[0])

    uf = UnionFind(names)
    mst = []

    for dist, a, b, c in edges:
        if uf.union(a, b):
            mst.append((a, b, c, dist))

    return mst


def clusters_from_mst(mst_edges, corr_threshold=0.5):
    # Form clusters by removing low-correlation edges.
    graph = {}
    nodes = set()

    for a, b, c, _ in mst_edges:
        nodes.add(a)
        nodes.add(b)
        if c >= corr_threshold:
            # cite the below 2 lines from AI
            graph.setdefault(a, []).append(b)
            graph.setdefault(b, []).append(a)

    for n in nodes:
        graph.setdefault(n, [])

    visited = set()
    groups = []

    # find connected components
    for n in nodes:
        if n not in visited:
            stack = [n]
            comp = []
            visited.add(n)

            while stack:
                x = stack.pop()
                comp.append(x)
                for y in graph[x]:
                    if y not in visited:
                        visited.add(y)
                        stack.append(y)

            groups.append(comp)

    return groups

# 3. Visualization

def plot_mst_graph(mst_edges, clusters):
    # Draw MST on a circle layout.
    nodes = [n for g in clusters for n in g]
    N = len(nodes)

    # cite the below 2 lines from AI
    angles = np.linspace(0, 2*np.pi, N, endpoint=False)
    pos = {nodes[i]: (np.cos(angles[i]), np.sin(angles[i])) for i in range(N)}

    colors = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple',
              'tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan']

    node_color = {}
    for idx, g in enumerate(clusters):
        for name in g:
            # cite the below line from AI
            node_color[name] = colors[idx % len(colors)]

    plt.figure(figsize=(6,6))

    for a, b, _, _ in mst_edges:
        x1, y1 = pos[a]
        x2, y2 = pos[b]
        plt.plot([x1, x2], [y1, y2], color='lightgray', linewidth=0.6)

    for name, (x, y) in pos.items():
        # edited the below 2 line with AI's help
        plt.scatter(x, y, color=node_color[name], s=45)
        plt.text(x, y, name, fontsize=7, ha='center', va='center')

    plt.title("MST (Clustered)")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_cluster_time_series(price_df, clusters):
    # Plot price histories for each cluster.
    for i, group in enumerate(clusters):
        plt.figure(figsize=(8,4))
        for ticker in group:
            if ticker in price_df.columns:
                plt.plot(price_df.index, price_df[ticker], label=ticker)

        plt.title(f"Cluster {i+1}")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.tight_layout()
        plt.show()


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Compute log returns and correlation

def compute_log_returns(price_df):
    # Return log price differences for each stock.
    log_prices = np.log(price_df)
    returns = log_prices.diff().dropna()
    return returns


def compute_correlation_matrix(returns_df):
    # Return correlation matrix of returns.
    return returns_df.corr()

# 2. Union-Find and MST construction

# refer to Lauren's code in lecture and HW
class UnionFind:
    def __init__(self, items):
        self.parent = {x: x for x in items}
        self.rank = {x: 0 for x in items}

    def find(self, x):
        # path compression
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a, b):
        # union by rank
        root_a = self.find(a)
        root_b = self.find(b)

        if root_a == root_b:
            return False

        if self.rank[root_a] < self.rank[root_b]:
            self.parent[root_a] = root_b
        elif self.rank[root_a] > self.rank[root_b]:
            self.parent[root_b] = root_a
        else:
            self.parent[root_b] = root_a
            self.rank[root_a] += 1

        return True


def build_mst_from_corr(corr_matrix):

    # Build an MST using distance = 1 - correlation.
    names = list(corr_matrix.columns)
    edges = []

    # collect pairwise distances
    for i in range(len(names)):
        for j in range(i + 1, len(names)):
            a = names[i]
            b = names[j]

            # cite the below line from AI
            c = corr_matrix.loc[a, b]
            dist = 1 - c
            edges.append((dist, a, b, c))

    # Kruskal: sort by distance
    edges.sort(key=lambda x: x[0])

    uf = UnionFind(names)
    mst_edges = []

    for dist, a, b, c in edges:
        if uf.union(a, b):
            mst_edges.append((a, b, c, dist))

    return mst_edges

# 3. Clustering from MST

def clusters_from_mst(mst_edges, corr_threshold=0.5):

    # Form clusters by removing edges whose correlation is below corr_threshold.
    graph = {}
    nodes = set()

    for a, b, c, dist in mst_edges:
        nodes.add(a)
        nodes.add(b)
        if c >= corr_threshold:
            # edited the below 2 lines with AI
            graph.setdefault(a, []).append(b)
            graph.setdefault(b, []).append(a)

    for n in nodes:
        graph.setdefault(n, [])

    visited = set()
    clusters = []

    for start in nodes:
        if start in visited:
            continue

        stack = [start]
        visited.add(start)
        comp = []

        while stack:
            x = stack.pop()
            comp.append(x)
            for y in graph[x]:
                if y not in visited:
                    visited.add(y)
                    stack.append(y)

        clusters.append(comp)

    return clusters

# 4. Cluster summary statistics (new)

def summarize_clusters(corr_matrix, clusters):
    """
    For each cluster, compute some simple stats.
    Returns a DataFrame with:
      - size
      - mean correlation
      - min correlation
      - max correlation
    """
    rows = []
    for i, group in enumerate(clusters):
        # submatrix for this cluster
        # cite the below line from AI
        sub = corr_matrix.loc[group, group]

        # drop diagonal
        # edited the below 2 lines with AI
        mask = ~np.eye(len(sub), dtype=bool)
        vals = sub.values[mask]

        if len(vals) == 0:
            avg_corr = np.nan
            min_corr = np.nan
            max_corr = np.nan
        else:
            avg_corr = float(np.nanmean(vals))
            min_corr = float(np.nanmin(vals))
            max_corr = float(np.nanmax(vals))

        rows.append(
            {
                "cluster_id": i,
                "size": len(group),
                "mean_corr": avg_corr,
                "min_corr": min_corr,
                "max_corr": max_corr,
            }
        )

    # cite the below line from AI
    summary_df = pd.DataFrame(rows)
    return summary_df

# 5. Visualization: MST + time series

def _build_circle_layout(clusters):
    # Helper: place nodes evenly on a circle.
    nodes = [n for group in clusters for n in group]
    n = len(nodes)
    # cite the below line from AI
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)

    positions = {}
    for i, name in enumerate(nodes):
        # cite the below 2 lines from AI
        x = np.cos(angles[i])
        y = np.sin(angles[i])
        positions[name] = (x, y)
    return positions


def plot_mst_graph(mst_edges, clusters, title="MST (clustered)"):
    # Draw MST on a circle layout, colored by cluster.
    pos = _build_circle_layout(clusters)

    palette = [
        "tab:blue",
        "tab:orange",
        "tab:green",
        "tab:red",
        "tab:purple",
        "tab:brown",
        "tab:pink",
        "tab:gray",
        "tab:olive",
        "tab:cyan",
    ]

    color_map = {}
    for idx, group in enumerate(clusters):
        # cite the below line from AI
        color = palette[idx % len(palette)]
        for name in group:
            color_map[name] = color

    plt.figure(figsize=(6, 6))

    # draw edges
    for a, b, c, dist in mst_edges:
        x1, y1 = pos[a]
        x2, y2 = pos[b]
        plt.plot([x1, x2], [y1, y2], color="lightgray", linewidth=0.6)

    # draw nodes
    for name, (x, y) in pos.items():
        # edited the below 2 lines with AI
        plt.scatter(x, y, color=color_map.get(name, "black"), s=45)
        plt.text(x, y, name, fontsize=7, ha="center", va="center")

    plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


def plot_cluster_time_series(price_df, clusters):
    # Plot the price history for each cluster.
    for i, group in enumerate(clusters):
        plt.figure(figsize=(8, 4))
        for name in group:
            if name in price_df.columns:
                plt.plot(price_df.index, price_df[name], label=name)

        plt.title(f"Cluster {i + 1}")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.tight_layout()
        plt.show()

# 6. Heatmap visualization  (new)

def plot_correlation_heatmap(corr_matrix, title="Correlation heatmap", labels=True):
    # Simple heatmap of the correlation matrix.

    # cite the below line from AI
    plt.figure(figsize=(6, 5))
    plt.imshow(corr_matrix.values, origin="lower",
               cmap="coolwarm", vmin=-1, vmax=1)
    plt.colorbar(label="Correlation")

    if labels:
        names = list(corr_matrix.columns)
        ticks = range(len(names))
        plt.xticks(ticks, names, rotation=90, fontsize=6)
        plt.yticks(ticks, names, fontsize=6)
    else:
        plt.xticks([])
        plt.yticks([])

    plt.title(title)
    plt.tight_layout()
    plt.show()


def plot_clustered_heatmap(corr_matrix, clusters, title="Clustered correlation"):
    # Reorder the correlation matrix by cluster membership.
    # Draw a heatmap to show block structure.

    ordered = []
    for group in clusters:
        for name in group:
            if name in corr_matrix.columns:
                ordered.append(name)

    # just in case some names were not in any cluster list
    for name in corr_matrix.columns:
        if name not in ordered:
            ordered.append(name)

    reordered = corr_matrix.loc[ordered, ordered]
    plot_correlation_heatmap(reordered, title=title, labels=True)

# 7. Rolling-window analysis  (new)

def make_time_windows(price_df, window_size, step):
    # Build a list of (start_idx, end_idx) windows.
    # window_size and step are in number of rows.

    n = len(price_df)
    windows = []
    start = 0

    while start + window_size <= n:
        end = start + window_size
        windows.append((start, end))
        start += step

    return windows


def run_single_window(price_df, start, end, corr_threshold=0.6):
    # Run the full pipeline on one time window.

    # cite the below 6 lines from AI
    window_prices = price_df.iloc[start:end]
    returns = compute_log_returns(window_prices)
    corr = compute_correlation_matrix(returns)
    mst = build_mst_from_corr(corr)
    clusters = clusters_from_mst(mst, corr_threshold=corr_threshold)
    summary = summarize_clusters(corr, clusters)

    return {
        "corr": corr,
        "mst": mst,
        "clusters": clusters,
        "summary": summary,
        "start": window_prices.index[0],
        "end": window_prices.index[-1],
    }


def rolling_window_analysis(price_df, window_size, step, corr_threshold=0.6):
    # Run the MST + clustering analysis over rolling windows.
    # Returns a list of window_results dicts.

    windows = make_time_windows(price_df, window_size, step)
    results = []

    for start, end in windows:
        # cite the below line from AI
        info = run_single_window(price_df, start, end,
                                 corr_threshold=corr_threshold)
        results.append(info)

    return results


def plot_window_summaries(window_results):
    # Quick line charts of cluster counts and average cluster size over time.

    if not window_results:
        return

    # edited the below line with AI
    dates = [w["end"] for w in window_results]
    num_clusters = [len(w["clusters"]) for w in window_results]
    avg_size = []

    for w in window_results:
        sizes = [len(c) for c in w["clusters"]]
        if sizes:
            avg_size.append(sum(sizes) / len(sizes))
        else:
            avg_size.append(0)

    plt.figure(figsize=(8, 3))
    plt.plot(dates, num_clusters, marker="o")
    plt.title("Number of clusters over time")
    plt.xlabel("Window end date")
    plt.ylabel("Cluster count")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 3))
    plt.plot(dates, avg_size, marker="o")
    plt.title("Average cluster size over time")
    plt.xlabel("Window end date")
    plt.ylabel("Avg cluster size")
    plt.tight_layout()
    plt.show()

# 8. Example driver function  (for testing)

# I wrote this test case with the help of AI.

def run_full_analysis(price_df, corr_threshold=0.6,
                      window_size=None, step=None):
    # Helper to run everything in one place for manual testing in a notebook.

    # full-period analysis
    full_returns = compute_log_returns(price_df)
    full_corr = compute_correlation_matrix(full_returns)
    full_mst = build_mst_from_corr(full_corr)
    full_clusters = clusters_from_mst(full_mst,
                                      corr_threshold=corr_threshold)
    full_summary = summarize_clusters(full_corr, full_clusters)

    print("Full-sample cluster summary:")
    print(full_summary)

    plot_mst_graph(full_mst, full_clusters, title="Full-period MST")
    plot_clustered_heatmap(full_corr, full_clusters,
                           title="Full-period clustered corr")
    plot_cluster_time_series(price_df, full_clusters)

    # optional rolling-window analysis
    if window_size is not None and step is not None:
        windows = rolling_window_analysis(
            price_df,
            window_size=window_size,
            step=step,
            corr_threshold=corr_threshold,
        )
        plot_window_summaries(windows)

        # show the first window's clustered heatmap
        first = windows[0]
        title = f"First window clustered corr: {first['start']} to {first['end']}"
        plot_clustered_heatmap(first["corr"], first["clusters"],
                               title=title)

    # return objects in case we want to inspect them
    return {
        "corr": full_corr,
        "mst": full_mst,
        "clusters": full_clusters,
        "summary": full_summary,
    }
